In [1]:
import numpy as np
import pymorphy2
import string

### 1. Из текста книги http://az.lib.ru/k/kipling_d_r/text_0070.shtml вырезать все лишнее не относящееся к тексту (лицензии и т.п.).

Скопировал тескт книги в блокнот text.txt

In [2]:
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

### 2. Найти top 100 самых часто используемые нормализованных слов в тексте.

Убираем знаки препинания с помощью модуля string

In [3]:
tt = str.maketrans(dict.fromkeys(string.punctuation))
text = text.translate(tt)

Нормализацию проведём с помощью морфологического анализатора pymorphy2

In [4]:
morph = pymorphy2.MorphAnalyzer()

In [5]:
def lemmatize(text):
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

In [6]:
lemmatize_text = lemmatize(text)

Все слова приведены в начальную форму, теперь необходимо убрать стоп-слова, загрузим их из nltk.corpus

In [7]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('russian'))

lemmatize_text_drop_stop = []
for word in lemmatize_text: 
    if word not in stopwords.words('russian'):
        lemmatize_text_drop_stop.append(word)

Для сопоставления слова и его частоты в тексте используем класс FreqDist (frequency distributions):

In [8]:
from nltk.probability import FreqDist
frequency_word_dist = FreqDist(lemmatize_text_drop_stop)

Для отбора топ-100 наиболее встречающихся в тексте слов есть метод most_common:

In [9]:
frequency_word_dist.most_common(100)

[('это', 313),
 ('сказать', 312),
 ('свой', 256),
 ('мауголь', 226),
 ('который', 200),
 ('человек', 195),
 ('мочь', 190),
 ('всё', 187),
 ('котик', 163),
 ('маленький', 148),
 ('весь', 144),
 ('слон', 132),
 ('знать', 120),
 ('джунгли', 118),
 ('голова', 118),
 ('время', 117),
 ('нагой', 114),
 ('большой', 111),
 ('мальчик', 109),
 ('багира', 102),
 ('волк', 98),
 ('бал', 95),
 ('риккитикки', 94),
 ('говорить', 92),
 ('тумая', 89),
 ('нога', 84),
 ('обезьяна', 83),
 ('место', 81),
 ('очень', 77),
 ('ещё', 76),
 ('шер', 76),
 ('хан', 76),
 ('глаз', 75),
 ('детёныш', 73),
 ('кал', 68),
 ('видеть', 67),
 ('стать', 67),
 ('дерево', 67),
 ('земля', 66),
 ('наш', 65),
 ('день', 64),
 ('стая', 63),
 ('твой', 61),
 ('смотреть', 61),
 ('акел', 61),
 ('морской', 59),
 ('каа', 58),
 ('слово', 56),
 ('мать', 54),
 ('заметить', 54),
 ('сторона', 54),
 ('молодой', 52),
 ('бояться', 51),
 ('змея', 51),
 ('ответить', 50),
 ('убить', 50),
 ('лошадь', 50),
 ('спросить', 49),
 ('отец', 48),
 ('год', 48)

Топ-100 слов получены, но необходимо отметить, что 101 слово и несколько ещё тоже могут встречаться в тексте 36 раз и тогда однозначно выделить топ-100 не получится.

In [10]:
for word_freq in frequency_word_dist.most_common():
    if word_freq[1] == 36:
        print(word_freq)
    elif word_freq[1] < 36:
        break

('хвост', 36)
('ребёнок', 36)
('посмотреть', 36)
('закон', 36)
('минута', 36)
('подняться', 36)
('остров', 36)
('берег', 36)


Как и ожидалось 36 раз в тексте встречается 8 слов, это означает, что однозначно можно выделить топ-96 слов (отсекая встречаемость 37 и более раз) или топ-104 (отсекая встречаемость 36 и более раз).

### 3. Найти все формы слова для top 10 самых часто используемые нормализованных слов в тексте.

Создадим словарь, где ключами будут нормализованные слова, а значениями - список различных форм слова встречающиеся в тексте

In [11]:
top_10_all_forms = {}
for word_freq in frequency_word_dist.most_common(10):
    top_10_all_forms[word_freq[0]] = []

Заполним словарь с помощью метода normal_form из pymorphy2

In [12]:
for word in set(text.lower().split()):
    if morph.parse(word)[0].normal_form in top_10_all_forms:
        top_10_all_forms[morph.parse(word)[0].normal_form].append(word)

In [13]:
for word, all_forms in top_10_all_forms.items():
    print(word, all_forms, sep=' - ')
    print()

это - ['этом', 'этим', 'этого', 'это']

сказать - ['скажет', 'сказать', 'скажите', 'сказано', 'скажешь', 'скажи', 'сказав', 'сказала', 'сказало', 'сказал', 'сказали', 'сказанные', 'скажу']

свой - ['своем', 'своей', 'своими', 'свой', 'своя', 'свое', 'своим', 'своему', 'своего', 'свою', 'своих', 'свои']

мауголь - ['маугли']

который - ['которой', 'который', 'которым', 'которое', 'котором', 'которую', 'которых', 'которого', 'которому', 'которая', 'которые']

человек - ['людьми', 'человека', 'человеком', 'людям', 'человеку', 'человек', 'людях', 'людей', 'люди']

мочь - ['могли', 'мог', 'можем', 'могут', 'могла', 'может', 'можете', 'могу', 'могло', 'можешь']

всё - ['все']

котик - ['котики', 'котику', 'котика', 'котикам', 'котиком', 'котиков', 'котик']

маленький - ['меньше', 'маленькие', 'маленькое', 'маленького', 'мала', 'маленькому', 'маленьком', 'маленький', 'маленькая', 'маленьким', 'мал', 'маленькими', 'маленькой', 'маленьких']



Стоит отметить, что имя "Маугли" встречается всего в одном тексте никакой алгоритм не сможет корректно с ним разобраться, если не внести его в словарь известных слов, что судя по всему в pymorphy2 не сделано.

### 4. Найти все имена собственные.

Для поиска имён собственных воспользуемся тегом Name из pymorphy2

In [14]:
thresh_hold = 0.2

proper_names = set()

for word in lemmatize_text_drop_stop:
    for p in morph.parse(word):
        if 'Name' in p.tag and p.score >= thresh_hold:
            proper_names.add(p.normal_form)

In [15]:
proper_names

{'абиссиния',
 'агата',
 'акел',
 'алла',
 'анна',
 'арра',
 'багир',
 'багира',
 'барак',
 'бента',
 'билли',
 'биллимула',
 'бутерина',
 'гапур',
 'георгий',
 'дельфина',
 'джидур',
 'диганга',
 'дик',
 'дхак',
 'кавнапур',
 'карнак',
 'карэта',
 'кедда',
 'кербайн',
 'кербайна',
 'керик',
 'керика',
 'кола',
 'кханивар',
 'кханхивар',
 'лев',
 'магдаля',
 'маила',
 'марешаля',
 'мила',
 'наген',
 'нагена',
 'надежда',
 'ната',
 'павел',
 'павлин',
 'паталамон',
 'першад',
 'пол',
 'радж',
 'ранна',
 'роза',
 'рой',
 'салаа',
 'самбхур',
 'сахиба',
 'тарана',
 'теодор',
 'тэдди',
 'удейпур',
 'усама',
 'фернандес',
 'хая',
 'хромуля',
 'хуан',
 'худж',
 'хукмхей',
 'хулла',
 'эрорулала',
 'эррула'}

Понижение степени уверенности алгоритма (thresh_hold) ниже 0.2 приводит к появлению мусора в итоговом "списке" имён собственных.